# 밑바닥 딥러닝
## Chapter 05 Part 1
## 오차역전파법 


오차를 역(반대 방향)으로 전파하는 방법 (backward propagation of errors).

앞 장에서는 신경망의 가중치 매개변수의 기울기 (=가중치 매개변수에 대한 손실 함수의 기울기)를 수치 미분으로 계산했음

수치 미분은 단순하고 구현하기도 쉬우나 계산 시간이 오래걸린다는 게 단점.

# 5.1 계산 그래프
# 5.2. 연쇄 법칙
# 5.3. 역전파
# 5.4. 단순한 계층 구현
# 5.5. 활성화 함수 계층 구현하기
# 5.6. Affine/Softmax 계층 구현하기
# 5.7. 오차역전파법 구현하기
# 5.8. 정리


### 5.4.1. 곱셈 계층

모든 계층은 forward()와 backward()라는 공통의 메서드(인터페이스)를 갖도록 구현한다. 
forward()는 순전파, backward()는 역전파를 처리한다. 

먼저 곱셈 계층을 구현해보자. 곱셈 계층은 MulLayer라는 이름의 클래스로 다음과 같이 구현 가능하다. 

In [1]:
# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/ch05/layer_naive.py 소스 참고
class MulLayer:
    def __init__(self): 
        #인스턴스 변수인 x와 y를 초기화한다. 
        #이 두 변수는 순전파 시의 입력 값을 유지하기 위해 사용한다.
        self.x = None
        self.y = None
    
    #forward에서는 x와 y를 인수로 받고 두 값을 곱해서 반환한다. 
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    #backward에서는 상류에서 넘어온 미분(dout)에 순전파 때의 값을 '서로 바꿔' 곱한 후 하류로 흘린다.'
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x 
        
        return dx, dy
    


MulLayer를 사용해서 앞에서 본 '사과 쇼핑'을 구현해보자. 
앞 절에서는 계산 그래프의 순전파와 역전파를 써서 [그림 5-16]과 같이 계산할 수 있다. 

In [3]:
# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/ch05/buy_apple.py 소스 참고
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price) # 220


220.00000000000003


각 변수에 대한 미분은 backward()에서 구할 수 있다. 

In [6]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("사과 가격 대한 미분은",dapple, "사과 개수에 대한 미분은",dapple_num, "소비세에 대한 미분은",dtax) # 2.2 110 200

사과 가격 대한 미분은 2.2 사과 개수에 대한 미분은 110.00000000000001 소비세에 대한 미분은 200



backward()의 호출 순서는 forward() 때와 반대이다. 
backward()가 받는 인수는 '순전파의 출력에 대한 미분'임을 주의. 

가령 mul_apple_layer 라는 곱셈 계층은 순전파때는 apple_price를 출력하지만
역전파에서는 apple_price 의 미분 값인 ```dapple_price```를 인수로 받는다. 
마지막으로 이 코드를 실행한 결과는 [그림5-16]의 결과와 일치한다. 


### 5.4.2. 덧셈 계층

덧셈 노드인 덧셈 계층을 구현해보자. 

In [7]:
class AddLayer:
    def __init__(self): 
        pass #pass = 아무 실행하지 않음
    
    def forward(self, x, y): #입력받은 두 인수 x, y를 더해서 반환한다. 
        out = x + y 
        return out

    def backward(self, dout): #상류에서 내려온 미분(dout)을 그대로 하류로 흘린다. 
        dx = dout * 1
        dy = dout * 1
        return dx, dy

이상의 덧셈 계층과 곱셈 계층을 사용하여 사과 2개와 귤 3개를 사는 [그림 5-17]의 상황을 구현해보자

In [8]:
# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/ch05/buy_apple.py 소스 참고
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파 : 필요한 계층을 만들어 순전파 메서드를 적절한 순서로 호출
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# 역전파 : 순전파와 반대 순서로 역전파 메서드를 호출하면 원하는 미분이 나옴
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price)) # 715
print("dApple:", dapple) # 2.2
print("dApple_num:", int(dapple_num)) # 110
print("dOrange:", dorange) # 3.3
print("dOrange_num:", int(dorange_num)) # 165
print("dTax:", dtax) # 650


price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


# 5.5. 활성화 함수 계층 구현하기

계산 그래프를 신경망에 적용해보자
신경망을 구성하는 층 (계층) 각각을 클래스 하나로 구현한다. 
우선은 활성화 함수인 ReLU와 Sigmoid계층을 구현한다. 

## 5.5.1. ReLU 계층

활성화 함수로 사용되는 ReLU의 수식은 다음과 같다. 

위 식에서 x에 대한 y의 미분은 아래 식과 같이 구한다. 순전파 때의 입력인 x가 0보다 크면 역전파는 상류의 값을 그대로 하류로 흘린다. 반면, 순전파 때 x 가 0 이하면 역전파 때는 하류로 신호를 보내지 않는다. (0을 보낸다). 

계산 그래프로는 [그림5-18] 처럼 그릴 수 있다. 


### 그림 5-18

코드로 구현

신경망 계층의 forward()와 backward()함수는 넘파이 배열을 인수로 받는다고 가정한다. ReLU 계층을 구현한 코드는 common/layers.py에 있다. 

In [1]:
# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/common/layers.py 소스 참고
class Relu:
    def __init__(self): 
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
        
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

ReLU클래스는 ```mask``` 라는 인스턴스 변수를 가진다. ```mask```는 True/False로 구성된 넘파이 배열로, 순전파의 입력인 x의 원소 값이 0 이하인 인덱스는 True, 그 외(0보다 큰 원소)는 False로 유지한다. 

예컨데 ```mask```변수는 다음 예와 같이 True/False로 구성된 넘파이 배열을 유지한다. 

In [2]:
import numpy as np
x = np.array([[1.0, 0.5], [-2.0, 3.0]])
print(x)

[[ 1.   0.5]
 [-2.   3. ]]


In [3]:
mask = (x<=0)
print(mask)

[[False False]
 [ True False]]


[그림 5-18]과 같이 순전파 때의 입력 값이 0 이하면 역전파 때의 값은 0이 되어야 한다. 그래서 역전파 때는 순전파 때 만들어 둔 mask를 써서 mask의 원소가 True인 곳에서는 상류에서 전파된 dout을 0으로 설정한다. 

## 5.5.2. Sigmoid 계층

다음은 시그모이드 함수 차례이다. 시그모이드 함수는 다음과 같다 (remember?)

[식5.9] 
\begin{equation*}
y = \frac{1}{1 + exp(-x)}
\end{equation*}

### 그림 5-19. Sigmoid계층의 계산 그래프 (순전파) 
<img src = "./image/sigmoid_all.png"/>

- 'exp' 노드 : y = exp(x)
- '/' 노드 : y = 1/x 


In [7]:
!ls ../image

Readme.md      decision.png   logic.png      xor.png
boolean.png    gates.png      sigmoid_cg.png


___

### <center> 1 단계 </center>



<img src = "./image/sigmoid1.png"/>

'/'노드를 미분하면 다음 식이 된다. 

\begin{equation*}
\frac{\partial y}{\partial x} = -\frac{1}{x^{2}}
\end{equation*}

\begin{equation*}
= - y^{2}
\end{equation*}

역전파 때는 상류의 예측값에 순전파의 출력을 제곱한 후 마이너스를 붙인 -y<sup>2</sup> 을 곱해서 하류로 전달

___

###  <center> 2 단계 </center>

<img src = "./image/sigmoid2.png"/>

'+' 노드는 상류의 값을 여과 없이 하류로 내보내는 게 다이다. 

___

### <center> 3 단계 </center>

<img src = "./image/sigmoid3.png">

'exp'노드는 y=exp(x)연산을 수행하며, 그 미분은 다음과 같다. 

\begin{equation*}
\frac{\partial y}{\partial x} = exp(x)
\end{equation*}

계산 그래프에서는 상류의 값에 순전파 때의 출력(이 예에서는 exp(-x))을 곱해 하류로 전파한다. 

___

###  <center> 4 단계 </center> <br>
[그림 5-20] Sigmoid 계층의 계산 그래프

<img src = "./image/sigmoid4.png">

'x' 노드는 순전파 때의 값을 '서로 바꿔' 곱한다. 이 예에서는 -1을 곱한다. 

___

그림 5-20와 같이 Sigmoid계층의 역전파를 계산 그래프로 완성한다. [그림 5-20]에서 보듯이 역전파의 최종 출력인 

\begin{equation*}
\frac{\partial L}{\partial y} y^{2} exp(-x)
\end{equation*} 값이 하류 노드로 전파된다. 이는 순전파의 입력 x와 출력 y만으로 계싼할 수 있다는 것을 알 수 있다. 
그래서 [그림 5-20]의 계산 그래프의 중간 과정을 모두 묶어 [그림5-21]처럼 단순한 'sigmoid' 노드 하나로 대체할 수 있다. 


[그림 5-20]의 계산 그래프와 [그림5-21]의 간소화 버전의 결과는 똑같으나 간소화 버전이 역전파 과정의 중간 계산들을 생략할 수 있기에 더 효율적이다. 또, 노드를 **그룹화**하여 Sigmoid계층의 세세한 내용을 노출하지 않고 입력과 출력에만 집중할 수 있다는 것도 중요한 포인트이다. 

또한, \begin{equation*}
\frac{\partial L}{\partial y} y^{2} exp(-x) \end{equation*} 는 다음처럼 정리해서 쓸 수 있다. 

\begin{equation*}
\frac{\partial L}{\partial y} y^{2} exp(-x) = \frac{\partial L}{\partial y} \frac{1} { (1+exp(-x))^{2}} exp(-x)
\end{equation*}

\begin{equation*}
= \frac{\partial L}{\partial y} \frac{1} { 1+exp(-x)} \frac{exp(-x)} {1+exp(-x)}
\end{equation*}

\begin{equation*}
= \frac{\partial L}{\partial y} y (1-y)
\end{equation*}

이처럼 Sigmoid 계층의 역전파는 순전파의 출력(y)만으로 계산할 수 있다. 

In [5]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
        
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx
    
#이 구현에서는 순전파의 출력을 인스턴스 변수 out에 보관했다가, 역전파 계산 때 그 값을 사용한다. 

## 5.6.1. Affine/Softmax 계층 구현하기 

## 5.6.1. Affine 계층

신경망의 순전파에서는 가중치 신호의 총합을 계산하기 때문에 행렬의 곱(넘파이에서는 np.dot())을 사용함 ("3.3 다차원 배열의 계산" 참고)
예를 들어 파이썬으로 다음과 같이 구현한 걸 기억할 것이다. 

In [6]:
X = np.random.rand(2)   # 입력
W = np.random.rand(2,3) # 가중치
B = np.random.rand(3)   # 편향

print(X.shape) # (2,)
print(W.shape) # (2, 3)
print(B.shape) # (3,)

Y = np.dot(X, W) + B

(2,)
(2, 3)
(3,)


여기서 X, W, B는 각각 형상이 (2,), (2,3), (3,)인 다차원 배열이다. 그러면 뉴런의 가중치 합은 Y = np.dot(X, W) + B처럼 계산한다. 그리고 이 Y를 활성화 함수로 변환해 다음 층으로 전파하는 것이 신경망 순전파의 흐름이다. 

행렬의 곱 계산은 대응하는 차원의 원소 수를 일치시키는게 핵심이다. 예를 들어 X와 W의 곱은 [그림 5-23]처럼 대응하는 차원의 원소 수를 일치시켜야 한다. 

[그림 5-23] 행렬의 곱에서는 대응하는 차원의 원소 수를 일치시킨다. 

<img src = "./image/matrix.png" width = 200/>

### 어파인 변환""affine transformation**
신경망의 순전파 때 수행하는 행렬의 곱은 기하학에서는 어파인 변환이라고 한다. 그래서 이 책에서는 어파인 변환을 수행하는 처리를 'Affine계층'이라는 이름으로 구현한다. 

[그림 5-24] Affine 계층의 계산 그래프

**변수가 행렬이고, 각 변수의 형상을 변수명 위에 표기함

<img src = "./image/affine_matrix.png">

[그림 5-24] 는 비교적 단순한 계산 그래프이다. 
단, **X, W, B가 행렬 (다차원 배열)이라는 점에 주의**할 것. 지금까지의 계산 그래프는 노드 사이에 '스칼라값'이 흘렀는데 반해, 이 예에서는 '행렬'이 흐르고 있다. 

이제 [그림 5-24]의 역전파에 대해 생각해보자. 
행렬을 사용한 역전파도 행렬의 원소마다 전개해보면 스칼라값을 사용한 지금까지의 계산 그래프와 같은 순서로 생각할 수 있다. 실제로 전개해보면 음 식이 도출된다. 

[식 5.13]

\begin{equation*}
\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} W^{T}
\end{equation*}

\begin{equation*}
\frac{\partial L}{\partial W} = X^{T} \frac{\partial L}{\partial Y}
\end{equation*}

여기서 W<sup>T</sup>의 T는 전치행렬을 뜻한다. 전치행렬은 W의 (i, j)위치의 원소를 (j, i)위치로 바꾼 것을 말한다. 수식으로는 다음과 같이 쓸 수 있다. 

\begin{equation*}
W =  \begin{vmatrix}
w_{11} w_{21} w_{31}\\
w_{12} w_{22} w_{32}\
\end{vmatrix}
\end{equation*}


전치행렬 W<sup>T</sup>

\begin{equation*}
W^{T} =  \begin{vmatrix}
w_{11} w_{12}\\
w_{21} w_{22}\\
w_{31} w_{32}\
\end{vmatrix}
\end{equation*}

식 5.13을 바탕으로 계산 그래프의 역전파를 구해본다. 결과는 [그림5-25]처럼 된다. 

### 그림 2-25 Affine 계층의 역전파

<img src = "./image/affine_back.png">

### 그림 2-26 행렬 곱('dot' node)의 역전파

**행렬의 대응하는 차원의 원소 수가 일치하도록 곱을 조립하여 구할 수 있다. 

<img src = "./image/affine_dot_product.png">

### 5.6.2. 배치용 Affine계층

지금까지 설명한 Affine계층은 입력 데이터로 X 하나만을 고려한 것이다. 이번 절에서는 데이터 N개를 묶어 순전파하는 경우, 즉 **배치용 Affine계층**을 생각해본다 (묶은 데이터를 '배치'라고 부른다) 

[그림 5-27] 배치용 Affine 계층의 계산 그래프

<img src = "./image/affine_batch.png"/>


기존과 다른 부분은 입력인 X 의 형상이 (N, 2)가 된 것 뿐이다. 그 뒤로는 지금까지와 같이 계산 그래프의 순서를 따라 순순히 행렬 계산을 하게 된다. 또, 역전파 때는 행렬의 형상에 주의하면 {\partial L}{\partial X}과 {\partial L}{\partial W}은 이전과 같이 도출할 수 있다. 

편향을 더할 때도 주의해야한다. 순전파 때의 편향 덧셈은 X*W에 대한 편향이 각 데이터에더해진다. 

예를 들어 N =2 (데이터가 2개)로 한 경우, 편향은 두 데이터 각각에 (각각의 계산 결과에) 더해진다. 

In [9]:
import numpy as np
X_dot_W = np.array([[0,0,0], [10,10,10]])
B = np.array([1,2,3])

In [10]:
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [11]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

순전파의 편향 덧셈은 각각의 데이터(1번째 데이터, 2번째 데이터...)에 더해진다. <= 이때문에 역전파 때는 각 데이터의 역전파 값이 편향의 원소에 모여야 한다. 

코드로는 다음과 같다. 

In [13]:
dY = np.array([[1,2,3], [4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [14]:
dB = np.sum(dY, axis = 0)

이 예에서는 데이터가 2개 (N =2)라고 가정한다. 
편향의 역전파는 그 두 데이터에 대한 미분을 데이터마다 더해서 구한다. 
그래서 np.sum()에서 0번째 축(데이터를 단위로 한 축)에 대해서 (axis = 0)의 총합을 구한다. 

이상의 Affine구현은 다음과 같다. 

**_참고로 common/layers.py 파일의 Affine 구현은 입력 데이터가 텐서 (4차원 데이터)인 경우도 고려한 것이라 다음 구현과는 약간 차이가 있다**

In [15]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

### 5.6.3. Softmax-with-Loss 계층

마지막으로 출력층에서 사용하는 소프트맥스 함수에 대해 알아보자. 

앞에서 말했듯이 소프트맥스 함수는 입력 값을 **정규화**하여 출력한다. 

예를 들어 손글씨 숫자 인식에서의 Softmax 계층의 출력은 [그림5-28]처럼 된다. 

[그림 5-28] 입력 이미지가 Affine 계층과 ReLU 계층을 통과하며 변환되고, 마지막 Softmax 계층에 의해서 10개의 입력이 정규화된다. 이 그림에서는 숫자 '0'의 점수는 5.3이며, 이것이 Softmax 계층에 의해서 0.008(0.8%)로 변환된다. 또, '2'의 점수는 10.1에서 0.991(99.1%)로 변환된다. 



[그림 5-28]과 같이 Softmax 계층은 입력 값을 정규화(출력의 합이 1이 되도록 변형)하여 출력한다. 

또한, 손글씨 숫자는 가짓수가 10개 (10클래스 분류)이므로 Softmax 계층의 입력은 10개가 된다. 

**_신경망에서 수행하는 작업은 "학습"과 "추론" 두 가지이다. 추론할 때는 일반적으로 Softmax 계층을 사용하지 않는다. 예컨데 [그림 5-28]의 신경망은 추론할 때는 마지막 Affine 계층의 출력을 인식 결과로 이용한다. 또한, 신경망에서 정규화하지 않는 출력 결과 [그림 5-28]에서는 Softmax앞의 Affine 계층의 출력을 점수(score)라고 한다. 
즉, 신경망 추론에서 답을 하나만 내는 경우에는 가장 높은 점수만 알면 되니 Softmax 계층은 필요 없다.반면 신경망을 학습할 때는 Softmax 계층이 필요하다._**

소프트맥스 계층을 구현할 때 손실함수인 *교차 엔트로피 오차*도 포함하여 "Softmax-with-Loss 계층'이라는 이름으로 구현한다. 먼저 Softmax-with-Loss 계층의 계산 그래프를 살펴보자 (그림 5-29). 

[그림 5-29] Softmax-with-Loss 계층의 계산 그래프 

[그림 5-30] Softmax-with-Loss 계층의 "간소화한" 계산 그래프 

<img src = "./image/softmax_with_loss.png">

설명 : 

1. 순전파 

     - 여기에서 Softmax 계층 (softmax 함수)와 Cross-entropy-error 계층 (Cross-entropy-error 함수)로 이루어진 3 클래스 분류를 가정하고 이전 계층에서 3개의 입력(점수)를 받는다. 
     - Softmax 계층은 입력 (a_1, a_2, a_3)을 정규화하여 (y_1, y_2, y_3)을 출력한다. 
     - Cross Entropy Error 계층은 Softmax의 출력(y_1, y_2, y_3)와 정답 레이블(t_1,t_2,t_3)를 받고, 이 데이터들로부터 손실 L을 출력한다. 

2. 역전파 

    - Softmax계층의 역전파는 (y_1 - t_1, y_2 - t_2, y_3 - t_3)라는 ("말끔한")결과를 출력 = softmax계층의 출력(y_n)과 정답 레이블(t_n)의 차이
    - 신경망의 역전파에서는 이 차이인 오차가 앞 계층에 전해짐 (=신경망 학습의 중요한 성질)
    - 신경망 학습의 목적 = 신경망의 출력(softmax의 출력)이 정답 레이블과 가까워지도록 가중치 매개변수의 값을 조정하는 것이기 때문에 신경망의 출력과 정답레이블의 오차를 효율적으로 앞 계층에 전달해야함.
    - 

In [16]:
# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/common/functions.py 소스 참고
# 3.5.2 소프트맥스 함수 구현시 주의점 참고
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 4.2.2. 교차 엔트로피 오차 참고
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 벡터)
    
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

### 5.7. 오차역전파법 구현하기

앞 절에서 구현한 계층을 조합하면 마치 레고 블록을 조합하듯 신경망을 구축할 수 있다. 이번 절에서는 지금까지 구현한 계층을 조합해서 신경망을 구축해보겠다. 

### 5.7.1. 신경망 학습의 전체 그림

다음은 신경망 학습의 순서이다. (4.5절 복습)

#### 전체 
신경망에서는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라고 한다. 신경망 학습은 다음과 같이 4단계로 수행한다. 

#### 1단계 : 미니배치

훈련 데이터 중 일부를 무작위로 가져온다. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실 함수 값을 줄이는 것이 목표이다. 

#### 2단계 : 기울기 산출  <= "오차역전파법"등장

미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시한다. 

#### 3단계: 매개변수 갱신

가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다. 

#### 1~3단계를 반복한다. 

** 앞 장에서는 2단계 기울기를 구하기 위해 수치 미분을 사용했지만 효율적이고 빠른 계산을 위해 오차역전파법을 이용한다. 


### 5.7.2. 오차역전파법을 적용한 신경망 구현하기 

여기에서는 2층 신경망을 TwoLayerNet클래스로 구현한다. 
앞 장의 "4.5 학습 알고리즘 구현하기"와 공통된 부분이 많지만 다른 점은 "계층"을 사용한다는 점이다(###표시). 계층을 사용함으로써 인식 결과를 얻는 처리(predict())와 기울기를 구하는 처리(gradient())계층의 전파만으로 동작이 이루어진다.


In [24]:
# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/ch05/two_layer_net.py 참고
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/common/functions.py
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/common/gradient.py 참고
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

#변수
#params = 딕셔너리 변수, 신경망의 매개변수를 보관

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01): #초기화 
        #size,output_size, weight_init_std = 인수는 앞에서부터 입력층 뉴런수, 은닉층 뉴런수, 츨력층 뉴런 수, 가중치 초기화 시 정규분포의 스케일
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        #1번째 층의 가중치
        self.params['b1'] = np.zeros(hidden_size) # 1번째 층의 편향
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)#2번째 층의 가중치
        self.params['b2'] = np.zeros(output_size) #2번째 층의 편향

        # layer = 계층 생성, 순서가 있는 딕셔너리 변수, 신경망의 계층을 보관
        self.layers = OrderedDict()                                           ###
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1']) ###
        self.layers['Relu1'] = Relu()                                         ###
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2']) ###
        #affine1->relu1->affine2와 같이 각 계층을 순서대로 유지
        self.lastLayer = SoftmaxWithLoss()                                    ###
        #신경망의 마지막 계층, 이 예시에서는 SoftmaxWithLoss()계층

    def predict(self, x): #예측(추론) 수행, 인수 x는 이미지 데이터 
        for layer in self.layers.values():                                    ###
            x = layer.forward(x)                                              ###
        
        return x
        
    # 손실함수의 값을 구함, x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    #정확도를 구한다. 
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # 가중치 매개변수의 기울기를 수치 미분 방식으로 구한다 (앞 장과 같음), x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    #가중치 매개 변수의 기울기를 오차역전파법으로 구한다.   
    def gradient(self, x, t):
        # forward
        self.loss(x, t)                      ###

        # backward
        dout = 1                             ###
        dout = self.lastLayer.backward(dout) ###
        
        layers = list(self.layers.values())  ###
        layers.reverse()                     ###
        for layer in layers:                 ###
            dout = layer.backward(dout)      ###

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads


### 5.7.3. 오차역전파법으로 구한 기울기 검증하기

지금까지 소개한 기울기를 구하는 방법은 1) 수치 미분을 써서 구하는 방법, 2) 해석적으로 수식을 풀어 구하는 방법(오차역전파법) 이렇게 2가지였다. 후자인 해석적 방법은 매개 변수가 많아도 효율적으로 계산이 가능하다. 

수치 미분은 오차역전파법을 정확히 구현했는지 확인하기 위해 필요하다. 수치 미분의 이점은 구현하기 쉬워서 버그가 숨어있기 어려운 반면, 오차역전파법은 구현하기 복잡해서 종종 실수를 하기도 한다. 따라서 이 둘의 구현의 차이를 이용하여 오차역전파법의 구현을 검증한다. 이처럼 두 방식으로 구한 기울기가 일치함을 확인하는 작업을 **기울기 확인**(gradient check)이라고 한다. 

기울기 확인은 다음과 같이 구현한다. 

In [18]:
!git clone https://github.com/WegraLee/deep-learning-from-scratch.git

Cloning into 'deep-learning-from-scratch'...
remote: Enumerating objects: 756, done.
remote: Total 756 (delta 0), reused 0 (delta 0), pack-reused 756
Receiving objects: 100% (756/756), 51.80 MiB | 2.53 MiB/s, done.
Resolving deltas: 100% (437/437), done.


In [19]:
import os

In [20]:
os.chdir("./deep-learning-from-scratch")

In [21]:
!ls

1_vs_2.png                ch04                      cover_image.jpg
LICENSE.md                ch05                      dataset
README.md                 ch06                      equations_and_figures.zip
ch01                      ch07                      map.png
ch02                      ch08
ch03                      common


In [ ]:

#import sys, os
#sys.path.append(os.pardir)
#import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape) # (60000, 784)
print(t_train.shape) # (60000, 10)

In [31]:
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
x_batch = x_train[:3]
t_batch = t_train[:3]

In [32]:
%timeit network.numerical_gradient(x_batch, t_batch)

8.96 s ± 381 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%timeit network.gradient(x_batch, t_batch)

231 µs ± 4.72 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


수치 미분(numerical_gradient)의 속도: 9.95초, 14.1초
오차역전법(gradient) 속도: 248 µs(0.000248초), 470 µs(0.000470초)
약 42,000배 속도 차이 남

In [29]:
# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/ch05/gradient_check.py 참고
# coding: utf-8
#import sys, os
#sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:3.6751404911006554e-10
b1:2.4094295146396582e-09
W2:4.554379620319809e-09
b2:1.3995554686840884e-07


결과 = 수치 미분과 오차역전파법으로 구한 기울기가 매우 작다고 말해줌. 
예를 들어, 1번째 층의 편향(B)오차는 9.7e-13(0.00000000000097)이다. 
이로써 오차역전파법이 실수 없이 구현했다고 믿을 수 있음 (수치 미분과 오차역전파법의 결과 오차가 0이 되는 일은 드물고, 올바르게 구현했다면 0에 아주 가까운 작은 값이 된다)

### 5.7.4. 오차역전파법을 사용한 학습 구현하기

오차역전파법으로 사용한 신경망 학습을 구현해보자. 
이전과 다른 부분은 기울기를 오차역전파법으로 구현한다는 점 뿐이다. 

In [30]:
# coding: utf-8
#import sys, os
#sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
#from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.13486666666666666 0.1321
0.9045 0.907
0.92475 0.9266
0.9368833333333333 0.9374
0.94305 0.9411
0.94875 0.9441
0.9527 0.9502
0.9593166666666667 0.9553
0.9625166666666667 0.9574
0.96585 0.9602
0.9665666666666667 0.9616
0.9675666666666667 0.9614
0.9715 0.9643
0.97365 0.9662
0.97505 0.9664
0.9760333333333333 0.9669
0.97745 0.9687


### 5.8. 정리 

- 이번 장에서는 ..

    1) 계산 그래프 (계산 과정을 시각적으로 보여주는 방법)에 대해 배움
    
    2) 계산 그래프의 노드는 국소적 계산으로 구성되며, 국소적 계산을 조합해 전체 계산을 구성한다. 
    
    3) 계산 그래프를 이용하여 신경망의 동작과 오차역전파법을 설명함
    
    4) 계산 과정을 "계층"이라는 단위로 구현함 (예:ReLU계층, Softmax-with-Loss계층, Affine 계층, Softmax계층)
    
    5) 모든 계층에서 구현되는 순전파(forward), 역전파(backward)에 대해 배움
    
    6) 계산 그래프의 순전파는 통상의 계산을 수행한다. 계산 그래프의 역전파로는 각 노드의 미분을 구할 수 있다. 
    
    7) 위의 동작을 계층으로 모듈화한 덕분에, 신경망의 구성 요소를 계층으로 구현하여 기울기를 효율적으로 계산할 수 있다. 
    
    8) 기울기 확인에서는 수치 미분과 오차역전파법의 결과를 비교함으로써 오차역전파법의 구현이 잘 되었는지 확인할 수 있다. 
    